In [2]:
# Core Python
import os
import sys
import math
import random
import time
from datetime import datetime

# Data manipulation
import numpy as np
import pandas as pd

#Text processing
import re

In [53]:
with open("the-verdict.txt", "r") as f:
    raw_text = f.read()
print("Total number of chars: ", len(raw_text))
print(raw_text[:99])

Total number of chars:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [54]:
process_raw = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
process_raw = [item.strip() for item in process_raw if item.strip()]
print(process_raw[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [72]:
all_words = sorted(set(process_raw))
vocab_size = len(all_words)
print(vocab_size)

1130


In [150]:
vocab = {token: i for i, token in enumerate(all_words)}

In [152]:
for token, index in vocab.items():
    print([token, index])
    if index == 30:
        break

['!', 0]
['"', 1]
["'", 2]
['(', 3]
[')', 4]
[',', 5]
['--', 6]
['.', 7]
[':', 8]
[';', 9]
['?', 10]
['A', 11]
['Ah', 12]
['Among', 13]
['And', 14]
['Are', 15]
['Arrt', 16]
['As', 17]
['At', 18]
['Be', 19]
['Begin', 20]
['Burlington', 21]
['But', 22]
['By', 23]
['Carlo', 24]
['Chicago', 25]
['Claude', 26]
['Come', 27]
['Croft', 28]
['Destroyed', 29]
['Devonshire', 30]


In [142]:
class basicTokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: token for token, i in vocab.items()}

    def encoder(self, text):
        process_raw = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        process_raw = [item.strip() for item in process_raw if item.strip()]
        ids = [self.str_to_int[s] for s in process_raw if s in self.str_to_int]
        return ids

    def decoder(self, ids):
        text = " ".join(self.int_to_str[i] for i in ids)
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


In [148]:
tokenizer = basicTokenizer(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride testnowordfromvocab."""

test_encoder = tokenizer.encoder(text)
test_encoder[:10]

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126]

In [149]:
test_decoder = tokenizer.decoder(test_encoder)
test_decoder

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'